In [ ]:
import oracledb, os, json
from google.cloud import secretmanager
import pandas as pd
import numpy as np
import paramiko
from io import StringIO

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    sftpkey = os.getenv('SFTPKEY'),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
# finn ut hva som står på serveren i bidrag området

keyfile = StringIO(oracle_secrets['sftpkey'])
mykey = paramiko.RSAKey.from_private_key(keyfile, password=np.nan)

# Open a transport
host,port = "a01drvl099.adeo.no",22
transport = paramiko.Transport((host,port))

# Auth    
username= "srv-dv-familie-airflow-sas"
transport.connect(username=username,pkey=mykey)

with paramiko.SFTPClient.from_transport(transport) as sftp:
    yup =sftp.listdir('./inbound/kildefiler/bidrag/')

yup

In [ ]:
def do_berm_etl():
    keyfile = StringIO(oracle_secrets['sftpkey'])
    mykey = paramiko.RSAKey.from_private_key(keyfile, password=np.nan)

    # Open a transport
    host,port = "a01drvl099.adeo.no",22
    transport = paramiko.Transport((host,port))

    # Auth    
    username= "srv-dv-familie-airflow-sas"
    transport.connect(username=username,pkey=mykey)

    for i in range(1, 13):
        if i < 10:
            i = '0'+str(i) #01, 02.....09
        with paramiko.SFTPClient.from_transport(transport) as sftp:
            #print("connected")
            #print(sftp.get_channel())#chdir('kildefiler/bidrag')
            sftp.get(f'./inbound/kildefiler/bidrag/BIDRAG_BERM_M2024{i}', f'../data/BIDRAG_BERM_M2024{i}')
            #print(sftp.listdir(path='.'))

    # Close
    if sftp: sftp.close()
    if transport: transport.close()

In [ ]:
do_berm_etl()

In [ ]:
def do_stonad_etl():
    keyfile = StringIO(oracle_secrets['sftpkey'])
    mykey = paramiko.RSAKey.from_private_key(keyfile, password=np.nan)

    # Open a transport
    host,port = "a01drvl099.adeo.no",22
    transport = paramiko.Transport((host,port))

    # Auth    
    username= "srv-dv-familie-airflow-sas"
    transport.connect(username=username,pkey=mykey)

    for i in range(1, 13):
        if i < 10:
            i = '0'+str(i) #01, 02.....09
        with paramiko.SFTPClient.from_transport(transport) as sftp:
            #print("connected")
            #print(sftp.get_channel())#chdir('kildefiler/bidrag')
            sftp.get(f'./inbound/kildefiler/bidrag/BIDRAG_STONAD_M2024{i}', f'../data/BIDRAG_STONAD_M2024{i}')
            #print(sftp.listdir(path='.'))

    # Close
    if sftp: sftp.close()
    if transport: transport.close()

In [ ]:
do_stonad_etl()

In [ ]:
%%bash

num = 0
for a in {1..12}; 
do
    if [[ $a -lt 10 ]]
    then
        num=0$a
    cat ../data/BIDRAG_STONAD_M2024$num | tr -d '\000' > ../data/BIDRAG_STONAD_M2024$num.txt;
    elif [[ $a -gt 9 ]]
    then
        num=$a
    cat ../data/BIDRAG_STONAD_M2024$num | tr -d '\000' > ../data/BIDRAG_STONAD_M2024$num.txt;
    fi
    done 

In [ ]:
barn_df_stonad = pd.read_excel('posisjoner/barn_stonad_pos.xlsx',header=0 )
barn_df_stonad.rename(columns={"Variabel-navn": "feltnavn", "Start-pos. i Bidrag-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

In [ ]:
barn_df_berm = pd.read_excel('./posisjoner/barn_berm_posisjoner.xlsx',header=0 )
barn_df_berm.rename(columns={"Forslag til Feltnavn": "feltnavn", "Start-pos. i BBM-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

In [ ]:
user = oracle_secrets['user'] + '[DVH_FAM_BB]'
dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])

In [ ]:
def hent_mottaker_berm_data(df_berm, df_barn_berm):

    mottaker_df_berm = pd.read_excel('./posisjoner/mottaker_berm_posisjoner.xlsx',header=0 )
    mottaker_df_berm.rename(columns={"Feltnavn": "feltnavn", "Start-pos. i BBM-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

    df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
    filt = (df_berm['REC_TYPE'] == '02')
    df_rec_type_2 = df_berm.loc[filt].copy()

    for index, row in mottaker_df_berm.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        df_rec_type_2[row['feltnavn']] = df_rec_type_2[0].apply(lambda x: x[start_pos:end_pos])

    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_mottaker_berm = df_rec_type_2[mottaker_df_berm['feltnavn']]
    df_mottaker_berm = df_mottaker_berm.replace(r'^\s*$', None, regex=True)

    # Filter df_mottaker based on SAKSNR in df_barn
    df_mottaker_berm = df_mottaker_berm[df_mottaker_berm['Saksnr'].isin(df_barn_berm['Saksnr'])]

    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            rows = [tuple(x) for x in df_mottaker_berm.values]
            cursor.executemany('''INSERT INTO FAM_BB_MOTTAKER_BERM(PERIODE,SAKSNR,REC_TYPE,FNR,VEDTDATO,SKATTFOR,INNTTYP1,INNTTYP2,INNTTYP3,INNTTYP4,INNTTYP5,KONTST
                                ,INNTBEL1,INNTBEL2,INNTBEL3,INNTBEL4,INNTBEL5,KONTSBEL,BTILRED,BARNETILBEL,BARNETILFORS)
                                VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21)''', rows)
            conn.commit()

In [ ]:
def hent_pliktig_berm_data(df_berm, df_barn_berm):

    pliktig_df_berm = pd.read_excel('./posisjoner/pliktig_berm_posisjoner.xlsx',header=0 )
    pliktig_df_berm.rename(columns={"Feltnavn": "feltnavn", "Start-pos. i BBM-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

    df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
    filt = (df_berm['REC_TYPE'] == '01')
    df_rec_type_1 = df_berm.loc[filt].copy()

    for index, row in pliktig_df_berm.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        df_rec_type_1[row['feltnavn']] = df_rec_type_1[0].apply(lambda x: x[start_pos:end_pos])

    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_pliktig_berm = df_rec_type_1[pliktig_df_berm['feltnavn']]
    df_pliktig_berm = df_pliktig_berm.replace(r'^\s*$', None, regex=True)

    # Filter df_mottaker based on SAKSNR in df_barn
    df_pliktig_berm = df_pliktig_berm[df_pliktig_berm['Saksnr'].isin(df_barn_berm['Saksnr'])]
    
    # Replace commas with dots and convert to numeric
    df_pliktig_berm['Antbarnh'] = df_pliktig_berm['Antbarnh'].str.replace(',', '.', regex=False).astype(float)

    
    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            rows = [tuple(x) for x in df_pliktig_berm.values]
            cursor.executemany('''INSERT INTO FAM_BB_PLIKTIGE_BERM(PERIODE,SAKSNR,REC_TYPE,FNR,VEDTDATO,ANTBARNH,INNTTYP1,INNTTYP2,INNTTYP3
                        ,INNTTYP4,INNTTYP5,KONTST,INNTBEL1,INNTBEL2,INNTBEL3,INNTBEL4,INNTBEL5,KONTSBEL,BTILRED,BARNETILBEL,BARNETILFORS,BIDREVNE,BOFORHOLD)
                                VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23)''', rows)
            conn.commit()

In [ ]:
def hent_barn_berm_data(df, i):
    df_berm = pd.read_csv(f"../data/BIDRAG_BERM_M2024{i}", skip_blank_lines=True, sep='\0', header=None)

    df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
    filt = (df_berm['REC_TYPE'] == '03')
    df_rec_type_3 = df_berm.loc[filt].copy()
    
    for index, row in barn_df_berm.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        #print(row['feltnavn'],start_pos, end_pos)
        df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
        
    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra barn_berm som inneholder navn på alle kolonner
    df_barn_berm = df_rec_type_3[barn_df_berm['feltnavn']]
    df_barn_berm = df_barn_berm.replace(r'^\s*$', None, regex=True)
    
    # Filter df_mottaker based on SAKSNR in df_barn
    df_barn_berm = df_barn_berm[df_barn_berm['Saksnr'].isin(df_barn_stonad['SAKSNR'])]
    
    df_barn_berm = df_barn_berm.loc[filt].copy()

    df_barn_berm['Bidrbe'] = pd.to_numeric(df_barn_berm['Bidrbe'].str.strip()) # convert this value '0      ' to nummeric 0

    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            rows = [tuple(x) for x in df_barn_berm.values]
            cursor.executemany('''INSERT INTO FAM_BB_BARN_BERM(PERIODE,SAKSNR,REC_TYPE,OBJNR,FNR,VEDTDATO,BIDRBE,BIDRBEL,BIDRTIL,BIDRES,HGBERM,UGBERM,AARSAK,SAMV,UNDERH,INNTTYP1,INNTTYP2,INNTTYP3,INNTTYP4,INNTTYP5,KONTST,
            INNTBEL1,INNTBEL2,INNTBEL3,INNTBEL4,INNTBEL5,KONTSBEL,BTILRED,HD,STDEKN,BTBEL,BTFAK,FORPL,TELLER,NEVNER,BPDELU,VIRKDATOBB,VEDTDATOFO,VIRKDATOFO,BPPROS,BTSKODE) 
                                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,
                                    :25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41)''',rows)
            conn.commit()
            
    hent_mottaker_berm_data(df_berm, df_barn_berm)
    
    hent_pliktig_berm_data(df_berm, df_barn_berm)

### barn_berm data

In [ ]:
%%time 

for i in range(1, 13):
    if i < 10:
        i = '0'+str(i)
    df_stonad = pd.read_csv(f"../data/BIDRAG_STONAD_M2024{i}.txt", skip_blank_lines=True, sep='\0', header=None)
    
    df_stonad['REC_TYPE'] = df_stonad[0].apply(lambda x: x[13:15])
    filt = (df_stonad['REC_TYPE'] == '03')
    df_rec_type_3 = df_stonad.loc[filt].copy()
    
    # her henter vi kolonne navn, posisjoner fra barn_df_stonad (excel fila) og så henter data fra filen BIDRAG_STONAD_M2023xx basert på kolonne og posisjoner
    for index, row in barn_df_stonad.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
        
    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_barn_stonad = df_rec_type_3[barn_df_stonad['feltnavn']]
    df_barn_stonad = df_barn_stonad.replace(r'^\s*$', None, regex=True)
    
    filt = (((df_barn_stonad['FORSKUDD'].fillna(0).astype(int) > 0)|(df_barn_stonad['BIDRAG_G'].fillna(0).astype(int) > 0) ))
    df_barn_stonad = df_barn_stonad.loc[filt].copy()

    ####### barn_berm basert på barn_stønad data (merk filteren)

    hent_barn_berm_data(df_barn_stonad, i)

In [ ]:
#### single periode

df_stonad = pd.read_csv(f"../data/BIDRAG_STONAD_M202301.txt", skip_blank_lines=True, sep='\0', header=None)

df_stonad['REC_TYPE'] = df_stonad[0].apply(lambda x: x[13:15])
filt = (df_stonad['REC_TYPE'] == '03')
df_rec_type_3 = df_stonad.loc[filt].copy()

# her henter vi kolonne navn, posisjoner fra barn_df_stonad (excel fila) og så henter data fra filen BIDRAG_STONAD_M2023xx basert på kolonne og posisjoner
for index, row in barn_df_stonad.iterrows():
    start_pos = row['start_pos'] - 1
    end_pos = start_pos + row['antall_pos']
    df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
    
# henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
df_barn_stonad = df_rec_type_3[barn_df_stonad['feltnavn']]
df_barn_stonad = df_barn_stonad.replace(r'^\s*$', None, regex=True)

filt = (((df_barn_stonad['FORSKUDD'].fillna(0).astype(int) > 0)|(df_barn_stonad['BIDRAG_G'].fillna(0).astype(int) > 0) ))
df_barn_stonad = df_barn_stonad.loc[filt].copy()


####### barn berm #########

df_berm = pd.read_csv(f"../data/BIDRAG_BERM_M202301", skip_blank_lines=True, sep='\0', header=None)

df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
filt = (df_berm['REC_TYPE'] == '03')
df_rec_type_3 = df_berm.loc[filt].copy()

for index, row in barn_df_berm.iterrows():
    start_pos = row['start_pos'] - 1
    end_pos = start_pos + row['antall_pos']
    #print(row['feltnavn'],start_pos, end_pos)
    df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
    
# henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra barn_berm som inneholder navn på alle kolonner
df_barn_berm = df_rec_type_3[barn_df_berm['feltnavn']]
df_barn_berm = df_barn_berm.replace(r'^\s*$', None, regex=True)

# Filter df_mottaker based on SAKSNR in df_barn
df_barn_berm = df_barn_berm[df_barn_berm['Saksnr'].isin(df_barn_stonad['SAKSNR'])]

df_barn_berm = df_barn_berm.loc[filt].copy()

df_barn_berm['Bidrbe'] = pd.to_numeric(df_barn_berm['Bidrbe'].str.strip())

######## Mottaker ##########

mottaker_df_berm = pd.read_excel('./posisjoner/mottaker_berm_posisjoner.xlsx',header=0 )
mottaker_df_berm.rename(columns={"Feltnavn": "feltnavn", "Start-pos. i BBM-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
filt = (df_berm['REC_TYPE'] == '02')
df_rec_type_2 = df_berm.loc[filt].copy()

for index, row in mottaker_df_berm.iterrows():
    start_pos = row['start_pos'] - 1
    end_pos = start_pos + row['antall_pos']
    df_rec_type_2[row['feltnavn']] = df_rec_type_2[0].apply(lambda x: x[start_pos:end_pos])

# henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
df_mottaker_berm = df_rec_type_2[mottaker_df_berm['feltnavn']]
df_mottaker_berm = df_mottaker_berm.replace(r'^\s*$', None, regex=True)

# Filter df_mottaker based on SAKSNR in df_barn
df_mottaker_berm = df_mottaker_berm[df_mottaker_berm['Saksnr'].isin(df_barn_berm['Saksnr'])]

########### pliktig berm ##########

pliktig_df_berm = pd.read_excel('./posisjoner/pliktig_berm_posisjoner.xlsx',header=0 )
pliktig_df_berm.rename(columns={"Feltnavn": "feltnavn", "Start-pos. i BBM-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
filt = (df_berm['REC_TYPE'] == '01')
df_rec_type_1 = df_berm.loc[filt].copy()

for index, row in pliktig_df_berm.iterrows():
    start_pos = row['start_pos'] - 1
    end_pos = start_pos + row['antall_pos']
    df_rec_type_1[row['feltnavn']] = df_rec_type_1[0].apply(lambda x: x[start_pos:end_pos])

# henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
df_pliktig_berm = df_rec_type_1[pliktig_df_berm['feltnavn']]
df_pliktig_berm = df_pliktig_berm.replace(r'^\s*$', None, regex=True)

# Filter df_mottaker based on SAKSNR in df_barn
df_pliktig_berm = df_pliktig_berm[df_pliktig_berm['Saksnr'].isin(df_barn_berm['Saksnr'])]

# Replace commas with dots and convert to numeric
df_pliktig_berm['Antbarnh'] = df_pliktig_berm['Antbarnh'].str.replace(',', '.', regex=False).astype(float)




In [ ]:
df_mottaker_berm

In [ ]:
# feil format (Kontsbel, Barnetilbel)
df_mottaker_berm.Barnetilfors.str.isnumeric().unique()

In [ ]:
non_numeric_rows = df_mottaker_berm[~df_mottaker_berm.Barnetilbel.str.isnumeric()]
non_numeric_rows.Barnetilbel.unique()

In [ ]:
df_pliktig_berm

In [ ]:
# feil format (Antbarnh)
df_pliktig_berm.Bidrevne.str.isnumeric().unique()

In [ ]:
non_numeric_rows = df_pliktig_berm[~df_pliktig_berm.Inntbel1.str.isnumeric()]
non_numeric_rows.Inntbel1.unique()

In [ ]:
def hent_barn_berm_data(df, i):
    df_berm = pd.read_csv(f"../data/BIDRAG_BERM_M2023{i}", skip_blank_lines=True, sep='\0', header=None)

    df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
    filt = (df_berm['REC_TYPE'] == '03')
    df_rec_type_3 = df_berm.loc[filt].copy()
    
    for index, row in barn_df_berm.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        #print(row['feltnavn'],start_pos, end_pos)
        df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
        
    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra barn_berm som inneholder navn på alle kolonner
    df_barn_berm = df_rec_type_3[barn_df_berm['feltnavn']]
    df_barn_berm = df_barn_berm.replace(r'^\s*$', None, regex=True)
    
    # Filter df_mottaker based on SAKSNR in df_barn
    df_barn_berm = df_barn_berm[df_barn_berm['Saksnr'].isin(df_barn_stonad['SAKSNR'])]
    
    df_barn_berm = df_barn_berm.loc[filt].copy()

    df_barn_berm['Bidrbe'] = pd.to_numeric(df_barn_berm['Bidrbe'].str.strip()) # convert this value '0      ' to nummeric 0

    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            rows = [tuple(x) for x in df_barn_berm.values]
            cursor.executemany('''INSERT INTO FAM_BB_BARN_BERM(PERIODE,SAKSNR,REC_TYPE,OBJNR,FNR,VEDTDATO,BIDRBE,BIDRBEL,BIDRTIL,BIDRES,HGBERM,UGBERM,AARSAK,SAMV,UNDERH,INNTTYP1,INNTTYP2,INNTTYP3,INNTTYP4,INNTTYP5,KONTST,
            INNTBEL1,INNTBEL2,INNTBEL3,INNTBEL4,INNTBEL5,KONTSBEL,BTILRED,HD,STDEKN,BTBEL,BTFAK,FORPL,TELLER,NEVNER,BPDELU,VIRKDATOBB,VEDTDATOFO,VIRKDATOFO,BPPROS,BTSKODE) 
                                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,
                                    :25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41)''',rows)
            conn.commit()
            
    hent_mottaker_berm_data(df_berm, df_barn_berm, conn)
    
    hent_pliktig_berm_data(df_berm, df_barn_berm, conn)

In [ ]:
def hent_pliktig_berm_data(df_berm, df_barn_berm, conn):

    pliktig_df_berm = pd.read_excel('./posisjoner/pliktig_berm_posisjoner.xlsx',header=0 )
    pliktig_df_berm.rename(columns={"Forslag til Feltnavn": "feltnavn", "Start-pos. i BBM-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

    df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
    filt = (df_berm['REC_TYPE'] == '01')
    df_rec_type_1 = df_berm.loc[filt].copy()

    for index, row in pliktig_df_berm.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        df_rec_type_1[row['feltnavn']] = df_rec_type_1[0].apply(lambda x: x[start_pos:end_pos])

    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_pliktig_berm = df_rec_type_1[pliktig_df_berm['feltnavn']]
    df_pliktig_berm = df_pliktig_berm.replace(r'^\s*$', None, regex=True)

    # Filter df_mottaker based on SAKSNR in df_barn
    df_pliktig_berm = df_pliktig_berm[df_pliktig_berm['SAKSNR'].isin(df_barn_berm['SAKSNR'])]

     with conn.cursor() as cursor:
        rows = [tuple(x) for x in df_pliktig_berm.values]
        cursor.executemany('''INSERT INTO FAM_BB_PLIKTIG_BERM('PERIODE','SAKSNR','REC_TYPE','FNR','VEDTDATO','ANTBARNH','INNTTYP1','INNTTYP2','INNTTYP3'
                    ,'INNTTYP4','INNTTYP5','KONTST','INNTBEL1','INNTBEL2','INNTBEL3','INNTBEL4','INNTBEL5','KONTSBEL','BTILRED','BARNETILBEL','BARNETILFORS','BIDREVNE','BOFORHOLD')
                            VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23)''', rows)
         conn.commit()
       

In [ ]:
df_barn_berm.Bidrbe.str.isnumeric().unique()

In [ ]:
non_numeric_rows = df_barn_berm[~df_barn_berm.Inntbel5.str.isnumeric()]
non_numeric_rows.Inntbel5.unique()

In [ ]:
filt = (df_barn_berm.Bidrbe != '000 0000')
df_barn_berm.loc[filt]

In [ ]:
# Strip leading/trailing spaces and replace non-numeric '0' with numeric 0
df_barn_berm['Bidrbe'] = df_barn_berm['Bidrbe'].str.strip()  # Remove leading/trailing spaces
df_barn_berm['Bidrbe'] = df_barn_berm['Bidrbe'].astype(int)#replace('0', '0.0')  # Ensure 0 is treated as numeric

# Convert the column to numeric, forcing errors to NaN, then replace NaN with 0
#df_barn_berm['Bidrbe'] = pd.to_numeric(df_barn_berm['Bidrbe'], errors='coerce').fillna(0)

# Check the result
df_barn_berm.Bidrbe.unique()


In [ ]:
def do_tl():

    user = oracle_secrets['user'] + '[DVH_FAM_BB]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])

    df_berm = pd.read_csv(f"data/BIDRAG_BERM_M202301", skip_blank_lines=True, sep='\0', header=None)

    #TODO
    df_berm['REC_TYPE'] = df_berm[0].apply(lambda x: x[13:15])
    filt = (df_berm['REC_TYPE'] == '03')
    df_rec_type_3 = df_berm.loc[filt].copy()

    for index, row in barn_df.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        #print(row['feltnavn'],start_pos, end_pos)
        df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
        
    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_barn = df_rec_type_3[barn_df['feltnavn']]
    df_barn = df_barn.replace(r'^\s*$', None, regex=True)

    # Filter df_mottaker based on SAKSNR in df_barn
    df_barn = df_barn[df_barn['Saksnr'].isin(df_barn_stonad['SAKSNR'])]
    
    df_barn = df_barn.loc[filt].copy()

    #(forskbis>0 or bidragg>0)

    #print(df_barn.head(5))

    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
            with conn.cursor() as cursor:
                rows = [tuple(x) for x in df_barn.values]
                cursor.executemany('''INSERT INTO FAM_BB_BARN_BERM(PERIODE,SAKSNR,REC_TYPE,OBJNR,FNR,VEDTDATO,BIDRBE,BIDRBEL,BIDRTIL,BIDRES,HGBERM,UGBERM,AARSAK,SAMV,UNDERH,INNTTYP1,INNTTYP2,INNTTYP3,INNTTYP4,INNTTYP5,KONTST,
    INNTBEL1,INNTBEL2,INNTBEL3,INNTBEL4,INNTBEL5,KONTSBEL,BTILRED,HD,STDEKN,BTBEL,BTFAK,FORPL,TELLER,NEVNER,BPDELU,VIRKDATOBB,VEDTDATOFO,VIRKDATOFO,BPPROS,BTSKODE) 
                                        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,
                                        :25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41)''',rows)
                conn.commit()
   

In [ ]:
do_tl()

In [ ]:
import os

for filename in os.listdir('../data'):
    if os.path.isfile(os.path.join('../data', filename)):
        os.remove(os.path.join('../data', filename))